# Satellite Image Analysis - Unsupervised Classification Pipeline

This notebook implements an unsupervised classification pipeline for satellite imagery using various feature extraction techniques and clustering algorithms.

## Pipeline Overview
1. Load and preprocess satellite images
2. Extract features (spectral, textural, edge, etc.)
3. Apply dimensionality reduction (PCA)
4. Perform clustering (K-means, MiniBatchKMeans)
5. Visualize and analyze results

In [4]:
import sys
import os

# Get the parent directory (root of the project)
parent_dir = os.path.dirname(os.getcwd())

# Add to sys.path if not already present
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Verify
print(f"Current working directory: {os.getcwd()}")
print(f"Parent directory added to sys.path: {parent_dir}")
print(f"sys.path: {sys.path}")

Current working directory: C:\Users\dell\Documents\GitHub\satellite-image-analysis\notebooks
Parent directory added to sys.path: C:\Users\dell\Documents\GitHub\satellite-image-analysis
sys.path: ['C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311', '', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\win32', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\dell\\Documents\\GitHub\\satellite-image-analysis']


In [5]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
from rasterio.plot import show
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skimage.feature import graycomatrix, graycoprops
from skimage.filters import sobel, laplace, gaussian
from skimage.color import rgb2gray
from skimage.exposure import rescale_intensity
import warnings
warnings.filterwarnings('ignore')

# Import functions from utils module
from utils.preprocessing import load_tif_images, flatten_bands, scale_features

# Load images
images, paths = load_tif_images("C:/Users/dell/Documents/GitHub/satellite-image-analysis/data/sample")

# Process each image
for img in images:
    flattened = flatten_bands(img)  # (H*W, C)
    scaled = scale_features(flattened)  # Normalized

ModuleNotFoundError: No module named 'utils.preprocessing'

## 1. Data Loading and Preprocessing

First, we'll load the satellite images from the data directory and examine their properties.

In [ ]:
# Define data directory path (adjust as needed for your environment)
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'sample'))
print(f"Loading images from: {data_dir}")

# Load raw images
images, paths = load_tif_images(data_dir)
print(f"Loaded {len(images)} images.")

# Display information about the first image
if len(images) > 0:
    sample_image = images[0]
    print(f"\nSample image shape: {sample_image.shape}")
    print(f"Number of bands: {sample_image.shape[0]}")
    print(f"Image dimensions (height × width): {sample_image.shape[1]} × {sample_image.shape[2]}")
    print(f"Data type: {sample_image.dtype}")
    print(f"Min value: {sample_image.min()}, Max value: {sample_image.max()}")
    
    # Display the filename of the sample image
    print(f"Sample image filename: {os.path.basename(paths[0])}")

## 2. Basic Image Visualization

Let's visualize the sample image using different band combinations.

In [ ]:
def display_rgb_composite(image, rgb_bands=[0, 1, 2], title="RGB Composite", figsize=(10, 8)):
    """Display an RGB composite image using specified bands."""
    # Extract the specified bands and transpose to (height, width, channels) for plotting
    rgb = np.dstack([image[band] for band in rgb_bands])
    
    # Normalize values to 0-1 range for display
    rgb_norm = np.zeros_like(rgb, dtype=np.float32)
    for i in range(3):
        band_min, band_max = np.percentile(rgb[:,:,i], (2, 98))
        rgb_norm[:,:,i] = np.clip((rgb[:,:,i] - band_min) / (band_max - band_min), 0, 1)
    
    # Display the image
    plt.figure(figsize=figsize)
    plt.imshow(rgb_norm)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Display RGB composite (assuming bands are in order: R, G, B, NIR)
if len(images) > 0:
    # True color composite (RGB)
    display_rgb_composite(images[0], rgb_bands=[0, 1, 2], title="True Color Composite (RGB)")
    
    # False color composite (NIR, R, G)
    display_rgb_composite(images[0], rgb_bands=[3, 0, 1], title="False Color Composite (NIR, R, G)")

## 3. Feature Extraction

We'll extract various features from the satellite images for clustering:
1. Spectral bands (RGB, NIR)
2. Spectral indices (NDVI, band ratios)
3. Textural features (GLCM)
4. Edge and shape features
5. Principal Component Analysis (PCA)

In [ ]:
def calculate_ndvi(image):
    """Calculate Normalized Difference Vegetation Index (NDVI).
    Assumes band order: R, G, B, NIR"""
    # Extract red and near-infrared bands
    red = image[0].astype(float)
    nir = image[3].astype(float)
    
    # Calculate NDVI: (NIR - Red) / (NIR + Red)
    # Add small epsilon to avoid division by zero
    epsilon = 1e-10
    ndvi = (nir - red) / (nir + red + epsilon)
    
    return ndvi

def calculate_band_ratios(image):
    """Calculate various band ratios.
    Assumes band order: R, G, B, NIR"""
    # Extract bands
    red = image[0].astype(float)
    green = image[1].astype(float)
    blue = image[2].astype(float)
    nir = image[3].astype(float)
    
    # Add small epsilon to avoid division by zero
    epsilon = 1e-10
    
    # Calculate ratios
    red_green_ratio = red / (green + epsilon)
    blue_green_ratio = blue / (green + epsilon)
    nir_red_ratio = nir / (red + epsilon)
    
    return red_green_ratio, blue_green_ratio, nir_red_ratio

# Calculate NDVI for the sample image
if len(images) > 0:
    ndvi = calculate_ndvi(images[0])
    
    # Visualize NDVI
    plt.figure(figsize=(10, 8))
    plt.imshow(ndvi, cmap='RdYlGn')
    plt.colorbar(label='NDVI')
    plt.title('Normalized Difference Vegetation Index (NDVI)')
    plt.axis('off')
    plt.show()
    
    # Calculate band ratios
    red_green_ratio, blue_green_ratio, nir_red_ratio = calculate_band_ratios(images[0])
    
    # Visualize one of the band ratios
    plt.figure(figsize=(10, 8))
    plt.imshow(nir_red_ratio, cmap='viridis')
    plt.colorbar(label='NIR/Red Ratio')
    plt.title('NIR/Red Band Ratio')
    plt.axis('off')
    plt.show()

In [ ]:
def extract_glcm_features(image, band_idx=0, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """Extract Gray-Level Co-occurrence Matrix (GLCM) textural features."""
    # Select a single band for GLCM calculation
    band = image[band_idx]
    
    # Rescale to 0-255 and convert to uint8 for GLCM calculation
    band_rescaled = rescale_intensity(band, out_range=(0, 255)).astype(np.uint8)
    
    # Calculate GLCM
    glcm = greycomatrix(band_rescaled, distances=distances, angles=angles, 
                        levels=256, symmetric=True, normed=True)
    
    # Calculate GLCM properties
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
    energy = greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0, 0]
    
    return contrast, dissimilarity, homogeneity, energy, correlation

def calculate_glcm_feature_maps(image, band_idx=0, window_size=15, step=15):
    """Calculate GLCM feature maps using a sliding window approach."""
    # Select a single band
    band = image[band_idx]
    height, width = band.shape
    
    # Initialize feature maps
    contrast_map = np.zeros((height // step, width // step))
    homogeneity_map = np.zeros_like(contrast_map)
    energy_map = np.zeros_like(contrast_map)
    
    # Calculate GLCM features for each window
    for i in range(0, height - window_size, step):
        for j in range(0, width - window_size, step):
            # Extract window
            window = band[i:i+window_size, j:j+window_size]
            
            # Rescale to 0-255 and convert to uint8
            window_rescaled = rescale_intensity(window, out_range=(0, 255)).astype(np.uint8)
            
            # Calculate GLCM
            glcm = greycomatrix(window_rescaled, distances=[1], angles=[0], 
                                levels=256, symmetric=True, normed=True)
            
            # Calculate GLCM properties
            contrast_map[i//step, j//step] = greycoprops(glcm, 'contrast')[0, 0]
            homogeneity_map[i//step, j//step] = greycoprops(glcm, 'homogeneity')[0, 0]
            energy_map[i//step, j//step] = greycoprops(glcm, 'energy')[0, 0]
    
    return contrast_map, homogeneity_map, energy_map

# Calculate GLCM feature maps for the sample image
if len(images) > 0:
    # Use a smaller window size for demonstration
    contrast_map, homogeneity_map, energy_map = calculate_glcm_feature_maps(images[0], band_idx=0, window_size=15, step=15)
    
    # Visualize GLCM feature maps
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    im0 = axes[0].imshow(contrast_map, cmap='hot')
    axes[0].set_title('GLCM Contrast')
    axes[0].axis('off')
    plt.colorbar(im0, ax=axes[0])
    
    im1 = axes[1].imshow(homogeneity_map, cmap='viridis')
    axes[1].set_title('GLCM Homogeneity')
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1])
    
    im2 = axes[2].imshow(energy_map, cmap='plasma')
    axes[2].set_title('GLCM Energy')
    axes[2].axis('off')
    plt.colorbar(im2, ax=axes[2])
    
    plt.tight_layout()
    plt.show()

In [ ]:
def extract_edge_features(image, band_idx=0):
    """Extract edge features using Sobel and Laplacian filters."""
    # Select a single band
    band = image[band_idx]
    
    # Apply Sobel filter for edge detection
    sobel_edges = sobel(band)
    
    # Apply Laplacian filter for edge detection
    laplacian_edges = np.abs(laplace(band))
    
    return sobel_edges, laplacian_edges

# Extract edge features for the sample image
if len(images) > 0:
    sobel_edges, laplacian_edges = extract_edge_features(images[0], band_idx=0)
    
    # Visualize edge features
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Original band
    axes[0].imshow(images[0][0], cmap='gray')
    axes[0].set_title('Original Band (Red)')
    axes[0].axis('off')
    
    # Sobel edges
    axes[1].imshow(sobel_edges, cmap='magma')
    axes[1].set_title('Sobel Edges')
    axes[1].axis('off')
    
    # Laplacian edges
    axes[2].imshow(laplacian_edges, cmap='viridis')
    axes[2].set_title('Laplacian Edges')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

## 4. Feature Combination and Dimensionality Reduction

Now we'll combine all the extracted features and apply PCA for dimensionality reduction.

In [ ]:
def extract_all_features(image):
    """Extract all features from an image and combine them."""
    # Get image dimensions
    bands, height, width = image.shape
    
    # Initialize feature array
    # Start with the original spectral bands
    features = np.zeros((height * width, bands))
    for b in range(bands):
        features[:, b] = image[b].flatten()
    
    # Calculate NDVI
    ndvi = calculate_ndvi(image)
    features = np.column_stack((features, ndvi.flatten()))
    
    # Calculate band ratios
    red_green_ratio, blue_green_ratio, nir_red_ratio = calculate_band_ratios(image)
    features = np.column_stack((features, 
                               red_green_ratio.flatten(),
                               blue_green_ratio.flatten(),
                               nir_red_ratio.flatten()))
    
    # Extract edge features
    sobel_edges, laplacian_edges = extract_edge_features(image, band_idx=0)
    features = np.column_stack((features, 
                               sobel_edges.flatten(),
                               laplacian_edges.flatten()))
    
    # Return the combined features
    return features

def apply_pca(features, n_components=3):
    """Apply PCA for dimensionality reduction."""
    # Standardize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    features_pca = pca.fit_transform(features_scaled)
    
    # Print explained variance ratio
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")
    
    return features_pca, pca

# Extract all features and apply PCA for the sample image
if len(images) > 0:
    # Extract features
    all_features = extract_all_features(images[0])
    print(f"Combined feature shape: {all_features.shape}")
    
    # Apply PCA
    features_pca, pca_model = apply_pca(all_features, n_components=3)
    print(f"PCA feature shape: {features_pca.shape}")
    
    # Visualize PCA components as an RGB image
    height, width = images[0].shape[1], images[0].shape[2]
    pca_rgb = features_pca.reshape(height, width, 3)
    
    # Normalize for visualization
    pca_rgb_norm = np.zeros_like(pca_rgb)
    for i in range(3):
        pca_rgb_norm[:,:,i] = (pca_rgb[:,:,i] - pca_rgb[:,:,i].min()) / (pca_rgb[:,:,i].max() - pca_rgb[:,:,i].min())
    
    plt.figure(figsize=(10, 8))
    plt.imshow(pca_rgb_norm)
    plt.title('PCA Components as RGB')
    plt.axis('off')
    plt.show()

## 5. Clustering

Now we'll apply K-means clustering to the extracted features and visualize the results.

In [ ]:
def apply_kmeans(features, n_clusters=5, random_state=42, use_minibatch=False, batch_size=1000):
    """Apply K-means clustering to the features."""
    # Standardize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Apply K-means or MiniBatchKMeans
    if use_minibatch:
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=batch_size)
    else:
        kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    
    # Fit and predict
    labels = kmeans.fit_predict(features_scaled)
    
    return labels, kmeans

def visualize_clusters(image, labels, title="K-means Clustering"):
    """Visualize clustering results."""
    # Reshape labels to image dimensions
    height, width = image.shape[1], image.shape[2]
    clustered_image = labels.reshape((height, width))
    
    # Visualize
    plt.figure(figsize=(10, 8))
    plt.imshow(clustered_image, cmap='tab20')
    plt.title(title)
    plt.axis('off')
    plt.colorbar(label='Cluster')
    plt.show()
    
    return clustered_image

# Apply K-means clustering to the PCA features
if len(images) > 0 and 'features_pca' in locals():
    # Try different numbers of clusters
    for k in [3, 5, 7]:
        # Apply K-means
        labels, kmeans_model = apply_kmeans(features_pca, n_clusters=k)
        
        # Visualize clusters
        clustered_image = visualize_clusters(images[0], labels, title=f"K-means Clustering (k={k})")

### 5.1 MiniBatchKMeans for Large Datasets

For larger datasets, we can use MiniBatchKMeans which is more efficient.

In [ ]:
# Apply MiniBatchKMeans to the original features (which might be larger)
if len(images) > 0 and 'all_features' in locals():
    # Apply MiniBatchKMeans
    labels_mb, kmeans_mb_model = apply_kmeans(all_features, n_clusters=5, use_minibatch=True, batch_size=1000)
    
    # Visualize clusters
    clustered_image_mb = visualize_clusters(images[0], labels_mb, title="MiniBatchKMeans Clustering (k=5)")

## 6. Comparison of Different Clustering Results

Let's compare the clustering results with different feature sets and parameters.

In [ ]:
def compare_feature_sets(image, k=5):
    """Compare clustering results with different feature sets."""
    # Get image dimensions
    bands, height, width = image.shape
    
    # 1. Original spectral bands only
    spectral_features = flatten_bands(image)
    spectral_labels, _ = apply_kmeans(spectral_features, n_clusters=k)
    
    # 2. Spectral bands + NDVI
    ndvi = calculate_ndvi(image)
    spectral_ndvi_features = np.column_stack((spectral_features, ndvi.flatten()))
    spectral_ndvi_labels, _ = apply_kmeans(spectral_ndvi_features, n_clusters=k)
    
    # 3. All features with PCA
    all_features = extract_all_features(image)
    features_pca, _ = apply_pca(all_features, n_components=10)
    pca_labels, _ = apply_kmeans(features_pca, n_clusters=k)
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Original RGB image
    rgb = np.dstack([image[0], image[1], image[2]])
    rgb_norm = np.zeros_like(rgb, dtype=np.float32)
    for i in range(3):
        band_min, band_max = np.percentile(rgb[:,:,i], (2, 98))
        rgb_norm[:,:,i] = np.clip((rgb[:,:,i] - band_min) / (band_max - band_min), 0, 1)
    
    axes[0].imshow(rgb_norm)
    axes[0].set_title('Original RGB')
    axes[0].axis('off')
    
    # Spectral bands only
    axes[1].imshow(spectral_labels.reshape((height, width)), cmap='tab20')
    axes[1].set_title('Spectral Bands Only')
    axes[1].axis('off')
    
    # Spectral bands + NDVI
    axes[2].imshow(spectral_ndvi_labels.reshape((height, width)), cmap='tab20')
    axes[2].set_title('Spectral Bands + NDVI')
    axes[2].axis('off')
    
    # All features with PCA
    axes[3].imshow(pca_labels.reshape((height, width)), cmap='tab20')
    axes[3].set_title('All Features with PCA')
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()

# Compare different feature sets
if len(images) > 0:
    compare_feature_sets(images[0], k=5)

### 6.1 Comparison of Different K Values

Let's compare the clustering results with different numbers of clusters (k values).

In [ ]:
def compare_k_values(image, features, k_values=[3, 5, 7, 10]):
    """Compare clustering results with different k values."""
    # Get image dimensions
    if len(image.shape) == 3:
        bands, height, width = image.shape
    else:
        height, width = image.shape
    
    # Create subplots
    fig, axes = plt.subplots(1, len(k_values), figsize=(5*len(k_values), 5))
    
    # Apply K-means with different k values
    for i, k in enumerate(k_values):
        # Apply K-means
        labels, _ = apply_kmeans(features, n_clusters=k)
        
        # Visualize clusters
        axes[i].imshow(labels.reshape((height, width)), cmap='tab20')
        axes[i].set_title(f'K = {k}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Compare different k values
if len(images) > 0 and 'features_pca' in locals():
    compare_k_values(images[0], features_pca, k_values=[3, 5, 7, 10])

### 6.2 Elbow Method for Optimal K Selection

We can use the Elbow Method to find the optimal number of clusters.

In [ ]:
def elbow_method(features, k_range=range(2, 11)):
    """Apply the Elbow Method to find the optimal number of clusters."""
    # Standardize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Calculate inertia for different k values
    inertia = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(features_scaled)
        inertia.append(kmeans.inertia_)
    
    # Plot the Elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertia, 'bo-')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)
    plt.show()

# Apply the Elbow Method
if len(images) > 0 and 'features_pca' in locals():
    # Use a sample of features for efficiency
    sample_size = min(10000, features_pca.shape[0])
    sample_indices = np.random.choice(features_pca.shape[0], sample_size, replace=False)
    features_sample = features_pca[sample_indices]
    
    elbow_method(features_sample)

## 7. Batch Processing Multiple Images

Now let's apply our pipeline to process multiple images in batch.

In [ ]:
def process_image_batch(images, paths, n_clusters=5, max_images=3):
    """Process a batch of images using the unsupervised classification pipeline."""
    # Limit the number of images to process
    n_images = min(len(images), max_images)
    
    # Process each image
    for i in range(n_images):
        print(f"\nProcessing image {i+1}/{n_images}: {os.path.basename(paths[i])}")
        
        # Extract features
        all_features = extract_all_features(images[i])
        print(f"  - Extracted {all_features.shape[1]} features")
        
        # Apply PCA
        features_pca, _ = apply_pca(all_features, n_components=10)
        
        # Apply K-means clustering
        labels, _ = apply_kmeans(features_pca, n_clusters=n_clusters)
        
        # Visualize results
        plt.figure(figsize=(15, 5))
        
        # Original RGB image
        plt.subplot(1, 2, 1)
        rgb = np.dstack([images[i][0], images[i][1], images[i][2]])
        rgb_norm = np.zeros_like(rgb, dtype=np.float32)
        for j in range(3):
            band_min, band_max = np.percentile(rgb[:,:,j], (2, 98))
            rgb_norm[:,:,j] = np.clip((rgb[:,:,j] - band_min) / (band_max - band_min), 0, 1)
        
        plt.imshow(rgb_norm)
        plt.title(f'Original RGB - {os.path.basename(paths[i])}')
        plt.axis('off')
        
        # Clustered image
        plt.subplot(1, 2, 2)
        height, width = images[i].shape[1], images[i].shape[2]
        plt.imshow(labels.reshape((height, width)), cmap='tab20')
        plt.title(f'K-means Clustering (k={n_clusters})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Process a batch of images
if len(images) > 1:
    process_image_batch(images, paths, n_clusters=5, max_images=3)

## 8. Conclusion

In this notebook, we have implemented a complete unsupervised classification pipeline for satellite imagery. The pipeline includes:

1. Loading and preprocessing satellite images
2. Extracting various features:
   - Spectral bands (RGB, NIR)
   - Spectral indices (NDVI, band ratios)
   - Textural features using GLCM (contrast, homogeneity, energy, etc.)
   - Edge and shape features using Sobel and Laplacian filters
3. Applying dimensionality reduction with PCA
4. Performing clustering with K-means and MiniBatchKMeans
5. Visualizing and comparing results with different feature sets and parameters

This pipeline can be used for various applications such as land cover classification, change detection, and environmental monitoring.

## 9. Future Work

Potential improvements and extensions to this pipeline:

1. Implement other clustering algorithms (DBSCAN, Hierarchical Clustering, etc.)
2. Add more advanced feature extraction techniques (Gabor filters, wavelet transforms)
3. Incorporate temporal information for time-series analysis
4. Implement supervised classification methods for comparison
5. Optimize the pipeline for large-scale processing
6. Add quantitative evaluation metrics for clustering quality